# Retrieval-Augmented Generation (RAG)
This is a notebook used for the 'Methods for Fintech and Artificial Intelligence in Finance'. The notebook experiment in different ways of adding valuable information to the answer generated by an LLM (retrieval-augmented generation, few shot learning). This notebook does not consider fine-tuning LLM as this typically comes with significant cost.

# Setting up the environment

https://community.anaconda.cloud/t/how-do-i-use-an-existing-environment-on-a-new-computer/55641

Load environment with conda
* cd ~/affine

* conda env create -f environment.yml

(If fist time, install a kernel)

* python -m ipykernel install --user --name=**kernel-name**


Remove kernel from system
* jupyter kernelspec list
* jupyter kernelspec uninstall **kernel-name**

Example code:

conda env create -n affine-project-env -f environment.yml

conda activate affine-project-env

python -m ipykernel install --user --name=affine-project-kernel


## Additional notes:
Create venv able with kernel:
* conda env export > environment.yml

In [1]:
%pip --version

pip 24.2 from /opt/miniconda3/envs/affine-project-env/lib/python3.12/site-packages/pip (python 3.12)
Note: you may need to restart the kernel to use updated packages.


In [3]:
#note: first we will use %pip install to test --> when package should be included in the environment use conda install
#dotenv
%pip install python-dotenv

# langchain set-up packages
%pip install --upgrade --quiet langchain
%pip install -qU "langchain-chroma>=0.1.2"
%pip install --upgrade --quiet  langchain-google-genai
%pip install --upgrade --quiet langchain-openai
%pip install --upgrade --quiet langchain-unstructured
# %pip install "unstructured[all-docs]"
#CHANGED BUT NOT TESTED YET
%pip install --upgrade --quiet unstructured-client
%pip install unstructured

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached unstructured-0.15.13-py3-none-any.whl.metadata (29 kB)
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached lxml-5.3.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.8 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata 

In [4]:
%pip freeze | grep langchain

langchain==0.3.2
langchain-chroma==0.1.4
langchain-core==0.3.9
langchain-google-genai==2.0.0
langchain-openai==0.2.2
langchain-text-splitters==0.3.0
langchain-unstructured==0.1.5
Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import dotenv_values
ENV = dotenv_values(".env")

In [ ]:
# from tutorial may be useful package to think about although langchain 0.2
# import os
# from langchain.vectorstores import Chroma
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain.schema.runnable import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
# from langchain.prompts import (
#     ChatPromptTemplate,
#     FewShotChatMessagePromptTemplate,
# )
# # from operator import itemgetter

# Prepare data
The data contains Apple’s:
* 10-Q for Q2-2024 (released on May 2, 2024) and Q3-2024* (released on August 1, 2024)
* Apples Q3 earnings call (released on August 1, 2024)
* 100 popular articles associated with Apple (released between September 4th  and October 5th, 2024)

The 10-Q .pdf files have been converted to json files using high resolution models as the pdf contain tables, more information:
* https://docs.unstructured.io/api-reference/how-to/choose-hi-res-model
* Also see: preprocessing_10-Q.py

In [2]:
# from langchain_unstructured import UnstructuredLoader
# loader = UnstructuredLoader('book-Copy1.txt')
# raw_doc = loader.load()

In [26]:
from langchain_text_splitters import CharacterTextSplitter
from os import listdir
from os.path import isfile, join
# segmenting the document into segments

def load_texts(input_folder_path):
  # Load an example document
  documents = []
  file_names = [f for f in listdir(input_folder_path) if isfile(join(input_folder_path, f))]
  print("file_names are: ")
  print(file_names)
  for file_name in file_names:
    file_path = "./" + input_folder_path + "/" + file_name
    with open(file_path, "r") as f:
      documents.append(f.read())

  text_splitter = CharacterTextSplitter(
      separator="\n\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len,
      is_separator_regex=False,
  )
  texts = text_splitter.create_documents(documents)
  return texts

In [23]:
# texts = load_texts('./data/final_data') #load_texts('./rawdata/apple_10-Q-Q2-2024-As-Filed.pdf')

In [7]:
from langchain_openai import AzureOpenAIEmbeddings

def text_embedding_3_small_azure():
  AZURE_OPENAI_ENDPOINT = ENV.get('AZURE_OPENAI_ENDPOINT')
  AZURE_OPENAI_API_KEY = ENV.get('AZURE_OPENAI_API_KEY')
  AZURE_OPENAI_API_VERSION = ENV.get('AZURE_OPENAI_API_VERSION')

  embedding_model = AzureOpenAIEmbeddings(
      model="text-embedding-3-small",
      # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
      azure_endpoint=AZURE_OPENAI_ENDPOINT, #If not provided, will read env variable AZURE_OPENAI_ENDPOINT
      api_key=AZURE_OPENAI_API_KEY, # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
      openai_api_version=AZURE_OPENAI_API_VERSION, # If not provided, will read env variable AZURE_OPENAI_API_VERSION
  )
  return embedding_model

In [ ]:
KDBAI_ENDPOINT = ENV.get('KDBAI_ENDPOINT')
KDBAI_TOKEN = ENV.get('KDBAI_TOKEN')

# Adding table description to table embedding

In [6]:
# DOCS:
# please be aware that chroma uses the embedding model in creating the vector store as well as the query
from langchain_chroma import Chroma
# Document Embedding with Chromadb

def create_vector_store(texts, embedding_model, persist_directory):
  vector_store = Chroma.from_documents(texts, embedding_model, persist_directory=persist_directory)
  return vector_store

def load_vector_store(embedding_model, persist_directory):
  vector_store = Chroma(embedding_function=embedding_model, persist_directory=persist_directory)
  return vector_store

#Chroma class https://python.langchain.com/api_reference/chroma/vectorstores/langchain_chroma.vectorstores.Chroma.html
def retrieve_relevant_chunks(chroma_vector_store, query):
  # Connection to query with Chroma indexing using a retriever
  retriever = chroma_vector_store.as_retriever(
      search_type="similarity",
      search_kwargs={'k':4}
  )
  
  chunks = retriever.invoke(query)
  return chunks

def print_chunks(chunks):
  for d in docs:
    print('--------------------------------NEW DOCS ----------------------------------------')
    print(d.page_content)

# Function to add all docs returned by retriever

# def format_docs(docs):
#   return "\n\n".join(doc.page_content for doc in docs)

In [24]:
# creating a new vector database
def create_vector_store_test_pipeline(document_path, persist_directory):
  texts = load_texts(document_path)
  embedding_model = text_embedding_3_small_azure()
  vector_store = Chroma.from_documents(texts, embedding_model, persist_directory=persist_directory)
  return vector_store

In [57]:
#basic_vector_store = create_vector_store_test_pipeline("./data/final_data", "./vector_stores/basic_vector_store")

In [18]:
basic_vector_store = load_vector_store(text_embedding_3_small_azure(), "./vector_stores/basic_vector_store")

In [27]:
enhanced_vector_store = create_vector_store_test_pipeline("./data/final_v2_data", "./vector_stores/enhanced_table_vector_store")

Created a chunk of size 1622, which is longer than the specified 1000
Created a chunk of size 2377, which is longer than the specified 1000
Created a chunk of size 1825, which is longer than the specified 1000
Created a chunk of size 1480, which is longer than the specified 1000
Created a chunk of size 2411, which is longer than the specified 1000
Created a chunk of size 3085, which is longer than the specified 1000
Created a chunk of size 1379, which is longer than the specified 1000
Created a chunk of size 1207, which is longer than the specified 1000
Created a chunk of size 2249, which is longer than the specified 1000
Created a chunk of size 2553, which is longer than the specified 1000
Created a chunk of size 1204, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of size 2303, which is longer than the specified 1000
Created a chunk of size 1316, which is longer than the specified 1000
Created a chunk of s

file_names are: 
['apple_10-Q-Q2-2024-As-Filed-Copy1.pdf.json.txt', 'apple_10-Q-Q3-2024-As-Filed-Copy1.pdf.json.txt', 'apple-Q3-2024-earnings-call.txt', 'apple-recent-news-Copy1.json.txt']


In [14]:
enhanced_vector_store = load_vector_store(text_embedding_3_small_azure(), "./vector_stores/enhanced_table_vector_store")

In [76]:
# retrieve_relevant_chunks(vector_store, "Does Pebblebee's news tracker work with Apple find my network")

# See difference in answer with context and no context

In [11]:
from langchain_google_genai import GoogleGenerativeAI
GEMINI_API_KEY = ENV.get('GEMINI_API_KEY')
# initializing the LLM

def compare_added_context_through_rag(query, vector_store, log = False):
  llm = GoogleGenerativeAI(model="models/gemini-1.5-flash", api_key=GEMINI_API_KEY)
  response_without_context = llm.invoke(query)

  chunks = retrieve_relevant_chunks(vector_store, query)
  context = "\n\n------\n\n".join([chunk.page_content for chunk in chunks]) #first_chunk.page_content
  rag_query = "\n\n".join([context, "\n\n", query])
  response_with_context = llm.invoke(rag_query)
  
  if log:
    print("response without context:\n")
    print(response_without_context + '\n\n')
    print("context:\n")
    print(context + "\n")
    print("response with context:\n")
    print(response_with_context)
  return response_without_context, response_with_context, context

# question 1

In [19]:
query = "Does Pebblebee's news tracker work with Apple find my network?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, basic_vector_store, log=True)

response without context:

Pebblebee devices, like the Pebblebee Clip, are designed to work with their own network and app. **They do not integrate with Apple's Find My network.**

Here's why:

* **Different Technology:** Pebblebee uses Bluetooth Low Energy (BLE) technology for tracking, while Apple's Find My network relies on a combination of Bluetooth, Ultra Wideband (UWB) technology, and crowd-sourced location data.
* **Closed Ecosystem:** Apple's Find My network is a closed ecosystem, only compatible with Apple devices and accessories that are specifically certified for the network. 

**However, Pebblebee offers a feature called "Lost & Found" which allows users to report a lost item and receive notifications if someone with the Pebblebee app finds it.** This network is separate from Apple's Find My network.

So, while Pebblebee devices won't work with Apple's Find My network, they have their own features for finding lost items. 



context:

Pebblebee’s new item trackers works wit

In [28]:
query = "Does Pebblebee's news tracker work with Apple find my network?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, enhanced_vector_store, log=True)

response without context:

Pebblebee's trackers do not work with Apple's Find My network. 

While Pebblebee trackers offer their own app and network for locating lost items, they are not compatible with Apple's Find My network. This means you cannot use the Find My app to locate your Pebblebee tracker.

If you're looking for a tracker that works with Find My, you should consider Apple's AirTags or other Find My compatible trackers. 



context:

Pebblebee’s new item trackers works with both Apple and Google 'Find My' networks
Apple’s Find My network and Google’s Find My Device are both smart tracking solutions to help us prevent losing our items, but devices made for one aren’t typically compatible with the other. However, Pebblebee is changing this by introducing its Pebblebee Un…

How to mirror your iPhone on macOS Sequoia
With macOS Sequoia and iOS 18, Apple has a handy new way to hop between devices while on desktop. iPhone Mirroring shows your phone’s screen on your computer; you 

## Local Graphrag
(affine-project-env) jovyan@c0a6df5293a0:~/affine$ python -m graphrag.query --root ./graphrag --method local "Does Pebblebee's news tracker work with Apple find my network?"

INFO: Vector Store Args: {}
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://affine-project-us-east.openai.azure.com', 'api_version': '2023-03-15-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o-mini', 'model_supports_json': True, 'tokens_per_minute': 20000, 'requests_per_minute': 100, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-3-small', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://affine-project-us-east.openai.azure.com', 'api_version': '2023-05-15', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-3-small', 'model_supports_json': None, 'tokens_per_minute': 10000, 'requests_per_minute': 50, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
Yes, Pebblebee's item trackers are compatible with Apple's Find My network. This partnership enhances the functionality of Apple's tracking services, allowing users to locate their lost items more effectively. The integration of Pebblebee's trackers into the Find My network signifies a technological collaboration aimed at improving device location services, which is crucial in today's tech-driven environment [Data: Reports (34); Entities (0, 19); Relationships (2)]. 

This collaboration not only benefits users by providing a reliable way to track their belongings but also raises important considerations regarding data privacy and security, as more devices become interconnected through tracking services [Data: Reports (34); Relationships (0, 1)].
(affine-project-env) jovyan@c0a6df5293a0:~/affine$ 

## Global GraphRAG

(affine-project-env) jovyan@c0a6df5293a0:~/affine$ python -m graphrag.query --root ./graphrag --method global "Does Pebblebee's news tracker work with Apple find my network?"


creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://affine-project-us-east.openai.azure.com', 'api_version': '2023-03-15-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o-mini', 'model_supports_json': True, 'tokens_per_minute': 20000, 'requests_per_minute': 100, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response:
## Compatibility of Pebblebee's News Tracker with Apple Find My Network

Pebblebee's item trackers indeed have compatibility with Apple's Find My network. This integration enhances the functionality of device location services, allowing users to locate their lost devices more effectively. The partnership between Pebblebee and Apple signifies a strong technological relationship that benefits users by providing a seamless experience in tracking their items [Data: Reports (34)].

## Additional Compatibility

In addition to its compatibility with Apple's Find My network, Pebblebee's item trackers are also compatible with Google's Find My Device service. This indicates a broader technological partnership that further enhances user experience and device management. However, it is important to note that the primary focus of the data provided emphasizes the relationship with Apple's network [Data: Reports (31)].

## Conclusion

In summary, Pebblebee's news tracker does work with Apple's Find My network, which may significantly improve the user experience in locating lost items. The compatibility with both Apple and Google services positions Pebblebee as a versatile option for users seeking reliable tracking solutions [Data: Reports (34, 31)].
(affine-project-env) jovyan@c0a6df5293a0:~/affine$ 

In [38]:
query = "Based on recent earning call is the company apple on track to reach its 2024 goals?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, basic_vector_store, log=True)

response without context:

I do not have access to real-time information, including recent earnings calls or company goals. Therefore, I cannot provide an answer to whether Apple is on track to reach its 2024 goals. 

To get this information, I recommend the following:

* **Check Apple's Investor Relations website:**  You can find transcripts of recent earnings calls, press releases, and other investor-related information on Apple's official website.
* **Read financial news articles:** Major business news outlets like Bloomberg, Reuters, and The Wall Street Journal will publish articles analyzing Apple's earnings calls and their implications for the company's future.
* **Consult with a financial advisor:** If you have specific questions about Apple's financial performance or goals, a financial advisor can provide you with personalized insights.

Remember, financial information is constantly changing, and it is important to stay up-to-date on the latest developments. 



context:

Apple

In [39]:
query = "Are sales of the IPhone 15 up compared to IPhone 14"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, basic_vector_store, log=True)

response without context:

I do not have access to real-time information, including sales figures for the iPhone 15. 

To get the most up-to-date information on iPhone 15 sales compared to iPhone 14 sales, I recommend checking:

* **Apple's official website:** They often release press releases or statements regarding sales performance.
* **Reputable tech news websites:** Sites like CNET, The Verge, TechCrunch, and others will report on sales figures and analysis.
* **Market research firms:** Companies like IDC, Gartner, and Canalys track smartphone sales data and publish reports.

Please note that sales figures can vary significantly depending on the source and the time period analyzed. 



context:

If you look at iPhone in particular for Greater China, the installed base set a record. We also in Mainland China set a June quarter record for upgraders and so that's a very strong signal and in fact from Kantar -- the survey from Kantar this quarter showed that iPhones were the top three

## Question 2

In [33]:
query = "How was Apple's Q3 performance compared to Q2"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, basic_vector_store, log=True)

response without context:

I do not have access to real-time information, including financial data like Apple's quarterly performance. To get the most up-to-date comparison between Apple's Q3 and Q2 performance, I recommend checking the following sources:

* **Apple's Investor Relations website:** This is the official source for Apple's financial information, including earnings releases and investor presentations.
* **Financial news websites:** Major financial news outlets like Bloomberg, Reuters, and The Wall Street Journal typically publish detailed analyses of Apple's quarterly earnings.
* **Market research firms:** Firms like Gartner, IDC, and Canalys provide independent analysis of the tech industry, including Apple's performance.

By consulting these sources, you can get a comprehensive understanding of how Apple's Q3 performance compared to Q2, including key metrics like revenue, earnings per share, and product sales. 



context:

iPhone net sales decreased during the second qu

In [35]:
query = "How was Apple's Q3 performance compared to Q2"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, enhanced_vector_store, log=True)

response without context:

I do not have access to real-time information, including financial data like Apple's quarterly performance. 

To get the most up-to-date comparison of Apple's Q3 performance to Q2, I recommend you:

* **Visit Apple's Investor Relations website:** You can find their official earnings releases and financial reports there.
* **Check reputable financial news sources:** Websites like Bloomberg, Reuters, The Wall Street Journal, and others will report on Apple's earnings and provide analysis.

Please note that comparing quarterly performance can be complex and requires looking at various factors like revenue, earnings per share, and specific product sales. 



context:

iPhone net sales decreased during the second quarter of 2024 compared to the second quarter of 2023 due to lower net sales of Pro models. Year-over-year iPhone net sales were relatively flat during the first six months of 2024.

Mac

Mac net sales increased during the second quarter and first six mo

### Local GraphRAG

SUCCESS: Local Search Response:
### Overview of Apple's Q3 Performance

Apple Inc. reported a strong performance in Q3 2024, showcasing significant growth compared to Q2 2024. The company achieved a record revenue of $85.8 billion in Q3, which marked a 5% increase year-over-year, despite facing challenges such as foreign exchange impacts [Data: Sources (53); Reports (36)].

### Revenue Breakdown

In Q3, Apple's revenue was bolstered by its Services segment, which reached an all-time high of $24.2 billion, reflecting a 14% growth compared to the previous year. This growth in Services was crucial, especially as it offset declines in hardware sales, particularly in the iPhone category, which saw a slight decrease of 1% year-over-year [Data: Sources (53); Reports (36)].

### Regional Performance

Regionally, Apple experienced varied performance. The Americas and Europe reported relatively flat net sales during Q2, but the June quarter saw record revenue in several countries, including Canada, Mexico, and various European nations. Notably, the Services segment reported higher net sales in both the Americas and Europe during Q2 compared to the same period in 2023 [Data: Entities (1, 125, 153); Relationships (288, 287, 34)].

### Challenges and Opportunities

Despite the overall positive outlook, Apple faced challenges in markets like Greater China and Japan, where net sales decreased due to lower iPhone and iPad sales. However, the company remains optimistic about long-term growth in these regions, particularly as the installed base of iPhones continues to grow [Data: Entities (154, 155); Reports (36)].

### Conclusion

In summary, Apple's Q3 performance demonstrated resilience and adaptability, with significant contributions from its Services segment helping to mitigate declines in hardware sales. The company’s strategic focus on innovation and expanding its service offerings positions it well for future growth, even amidst regional challenges [Data: Sources (53); Reports (36)].
(affine-project-env) jovyan@c0a6df5293a0:~/affine$ 

## Global GraphRAG

SUCCESS: Global Search Response:
## Overview of Apple's Q3 Performance

Apple's financial performance in Q3 2024 demonstrated a mix of achievements and challenges when compared to Q2 2024. The company reported record revenue during the June quarter, largely driven by strong sales of the iPhone 15 and services, indicating effective strategies and product offerings [Data: Reports (27, 28)]. However, this performance was contrasted by complexities in certain markets, particularly in Greater China and Japan, which may have impacted overall growth [Data: Reports (36, 37)].

## Key Highlights

### Revenue Growth
In Q3 2024, Apple experienced notable revenue growth, primarily attributed to the successful launch of the iPhone 16 Pro. This new product showcased significant technological advancements, contributing positively to the company's financial health [Data: Reports (9, 28)]. The services segment also continued to show impressive growth rates, reflecting a shift towards recurring revenue models that enhance user engagement and loyalty [Data: Reports (38)].

### Regional Variations
Despite the overall positive trajectory, Apple's performance varied significantly across regions. For instance, while the iPhone saw a decrease in net sales in Greater China and the Americas during Q2, there was an increase in Japan during Q3, highlighting regional disparities in market performance [Data: Reports (38)]. This mixed performance suggests that while some areas are thriving, others are facing challenges.

### Comparison with Q2
The performance in Q3 contrasts with Q2, where record revenue was also reported. However, the challenges faced in markets like Greater China and Japan during Q3 indicate a more complex landscape for maintaining growth across diverse economic environments [Data: Reports (36, 37)]. Additionally, the Mac product line showed resilience with growth in net sales during Q2 compared to the same period in 2023, but the overall sales trends in Europe revealed a mixed performance, with some segments experiencing declines [Data: Reports (37)].

## Conclusion

In summary, while Apple's Q3 2024 performance showcased strong revenue growth driven by new product launches and services, it also revealed challenges in specific markets and regional variations. The mixed trajectory between Q2 and Q3 highlights the complexities Apple faces in sustaining growth across its diverse product lines and global markets [Data: Reports (27, 28, 36, 37, 38, +more)].

In [37]:
query = "Could you return a table depicting the balance sheet for Apple's 3rd Quarter?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, basic_vector_store, log=True)

response without context:

I do not have access to real-time information, including financial data like Apple's balance sheet for specific quarters. To get the most up-to-date information on Apple's balance sheet for the 3rd quarter, I recommend checking the following resources:

* **Apple's Investor Relations website:** [https://investor.apple.com/](https://investor.apple.com/)
* **SEC EDGAR database:** [https://www.sec.gov/edgar/searchedgar/companysearch.html](https://www.sec.gov/edgar/searchedgar/companysearch.html) (Search for Apple Inc.)
* **Financial news websites:** Websites like Bloomberg, Reuters, and Yahoo Finance often publish financial statements for major companies.

These sources will provide you with the official balance sheet, including details on assets, liabilities, and equity. 



context:

Apple Inc. CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)

------

Apple Inc. CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)

------

Item 1. Financial Statements

Apple Inc

## Apple Inc. - Total Liabilities and Shareholders' Equity

| Period | Total Liabilities | Shareholders' Equity | Total Liabilities & Shareholders' Equity |
|---|---|---|---|
| Current Year | $264,904 | $74,194 | $337,411 |
| Prior Year | $290,437 | $62,146 | $352,583 | 


## Question 3

In [36]:
query = "Could you provide a percentage of the total liabilities vs shareholders' equity?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, enhanced_vector_store, log=True)

response without context:

Please provide me with the financial statements (balance sheet) of the company you are interested in.  I need the following information to calculate the percentage of total liabilities vs shareholders' equity:

* **Total Liabilities:** This is the sum of all the company's obligations to others.
* **Shareholders' Equity:** This represents the value of the company that belongs to its owners.

Once I have this information, I can calculate the percentage using the following formula:

**Percentage of Total Liabilities vs Shareholders' Equity = (Total Liabilities / (Total Liabilities + Shareholders' Equity)) * 100** 

For example, if a company has total liabilities of $100 million and shareholders' equity of $50 million:

**Percentage of Total Liabilities vs Shareholders' Equity = ($100 million / ($100 million + $50 million)) * 100 = 66.67%**

This means that 66.67% of the company's total assets are financed by debt, while the remaining 33.33% are financed by equit

In [86]:
query = "Could you provide a percentage of the total liabilities vs shareholders' equity?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, enhanced_table_vector_store, log=True)

response without context:

Please provide me with the financial statements (balance sheet) of the company you are interested in. I need the total liabilities and shareholders' equity figures to calculate the percentage. 

Once you provide the information, I can calculate the percentage for you using the following formula:

**Percentage of Total Liabilities to Shareholders' Equity = (Total Liabilities / Shareholders' Equity) * 100** 



context:

Shareholders’ equity:
This table presents the company's liabilities and shareholders' equity for two periods, likely representing the current year and the prior year. The table is divided into three main sections: Liabilities, Shareholders' Equity, and Total Liabilities and Shareholders' Equity.

**Liabilities** are categorized into current liabilities and non-current liabilities. Current liabilities include accounts payable, other current liabilities, deferred revenue, commercial paper, and term debt. Non-current liabilities include term debt 

(affine-project-env) jovyan@3300093a6113:~/affine$ python -m graphrag.query --root ./graphrag --method local "Could you provide a percentage of the total liabilities vs shareholders' equity?"



INFO: Vector Store Args: {}
[2024-10-06T23:30:04Z WARN  lance::dataset] No existing dataset at /home/jovyan/affine/graphrag/output/lancedb/entity_description_embeddings.lance, it will be created
creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://affine-project-us-east.openai.azure.com', 'api_version': '2023-03-15-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o-mini', 'model_supports_json': True, 'tokens_per_minute': 20000, 'requests_per_minute': 100, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
creating embedding llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_embedding", 'model': 'text-embedding-3-small', 'max_tokens': 4000, 'temperature': 0, 'top_p': 1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://affine-project-us-east.openai.azure.com', 'api_version': '2023-05-15', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'text-embedding-3-small', 'model_supports_json': None, 'tokens_per_minute': 10000, 'requests_per_minute': 50, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Local Search Response:
To calculate the percentage of total liabilities versus shareholders' equity for Apple Inc., we can use the most recent figures from the provided data.

### Total Liabilities and Shareholders' Equity

1. **Total Liabilities** (as of March 30, 2024): $264,904 million [Data: Sources (8)].
2. **Total Shareholders' Equity** (as of March 30, 2024): $66,708 million [Data: Sources (8)].

### Calculation

The formula to calculate the percentage of total liabilities compared to shareholders' equity is:

\[
\text{Percentage} = \left( \frac{\text{Total Liabilities}}{\text{Total Shareholders' Equity}} \right) \times 100
\]

Substituting the values:

\[
\text{Percentage} = \left( \frac{264,904}{66,708} \right) \times 100 \approx 397.5\%
\]

### Conclusion

The total liabilities of Apple Inc. are approximately **397.5%** of its shareholders' equity as of March 30, 2024. This indicates that the company has significantly more liabilities than equity, which is common in large corporations that leverage debt for growth and operations [Data: Sources (8)].
(affine-project-env) jovyan@3300093a6113:~/affine$ 

# Extraction from terminal
(affine-project-env) jovyan@3300093a6113:~/affine$ python -m graphrag.query --root ./graphrag --method global "Could you provide a percentage of the total liabilities vs shareholders' equity?"


creating llm client with {'api_key': 'REDACTED,len=32', 'type': "azure_openai_chat", 'model': 'gpt-4o-mini', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://affine-project-us-east.openai.azure.com', 'api_version': '2023-03-15-preview', 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': 'gpt-4o-mini', 'model_supports_json': True, 'tokens_per_minute': 20000, 'requests_per_minute': 100, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}
Warning: All map responses have score 0 (i.e., no relevant information found from the dataset), returning a canned 'I do not know' answer. You can try enabling `allow_general_knowledge` to encourage the LLM to incorporate relevant general knowledge, at the risk of increasing hallucinations.

SUCCESS: Global Search Response:
I am sorry but I am unable to answer this question given the provided data.

## Question 4

In [41]:
query = "Could you return a table depicting the balance sheet for Apple's 3rd Quarter?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, basic_vector_store, log=True)

response without context:

I do not have access to real-time information, including financial data like Apple's balance sheet. Therefore, I cannot provide you with a table depicting the balance sheet for Apple's 3rd Quarter. 

To find this information, I recommend you:

* **Visit Apple's Investor Relations website:** This is the official source for Apple's financial reports. You can find their latest earnings releases and 10-K reports, which contain detailed financial information. 
* **Consult reputable financial news sources:** Websites like Yahoo Finance, Google Finance, Bloomberg, and Reuters often publish summaries of company earnings reports, including balance sheet data.

Please note that financial information is constantly changing, so it's important to refer to the most up-to-date sources. 



context:

Apple Inc. CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)

------

Apple Inc. CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)

------

Item 1. Financial Statements

Apple I

In [40]:
query = "Could you return a table depicting the balance sheet for Apple’s 3rd Quarter?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, enhanced_vector_store, log=True)

response without context:

I do not have access to real-time information, including financial data like Apple's balance sheet for their 3rd quarter. 

To find this information, I recommend checking these sources:

* **Apple's Investor Relations Website:** You can find their quarterly earnings reports and 10-Q filings, which contain detailed financial statements, including the balance sheet.
* **SEC EDGAR Database:** The Securities and Exchange Commission (SEC) requires publicly traded companies to file their financial reports. You can search for Apple's filings on EDGAR.
* **Financial News Websites:** Websites like Yahoo Finance, Bloomberg, and Google Finance often publish summaries of companies' financial results, including their balance sheets.

Please note that the specific quarter you are interested in will be important when searching for this information. 



context:

Apple Inc. CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)

(In millions, except number of shares, which are re

### Local GraphRAG

I don't have the capability to create tables directly. However, I can summarize the key components of Apple Inc.'s balance sheet for the 3rd Quarter based on the available data.

### Apple Inc. Balance Sheet Summary for Q3 2024

#### Current Assets
- **Cash and Cash Equivalents:** $32,695 million
- **Marketable Securities:** $34,455 million
- **Accounts Receivable, Net:** $21,837 million
- **Vendor Non-Trade Receivables:** $19,313 million
- **Inventories:** $6,232 million
- **Other Current Assets:** $13,884 million
- ****Total Current Assets:** $128,416 million

#### Non-Current Assets
- **Marketable Securities:** $95,187 million
- **Property, Plant and Equipment, Net:** $43,546 million
- **Other Non-Current Assets:** $70,262 million
- **Total Non-Current Assets:** $208,995 million

#### Total Assets
- **Total Assets:** $337,411 million

#### Current Liabilities
- **Accounts Payable:** $45,753 million
- **Other Current Liabilities:** $57,298 million
- **Deferred Revenue:** $8,053 million
- **Commercial Paper:** (not specified)
- **Term Debt:** (not specified)
- **Total Current Liabilities:** (not specified)

### Global GraphRAG

I am sorry but I am unable to answer this question given the provided data.

## Question 5

In [44]:
query = "Based on recent earning call is the company apple on track to reach its 2024 goals?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, basic_vector_store, log=True)

response without context:

I do not have access to real-time information, including recent earnings calls or Apple's specific 2024 goals. 

To determine if Apple is on track to reach its 2024 goals, you would need to:

1. **Identify Apple's 2024 goals:** These are likely outlined in investor presentations, earnings calls, or public statements.
2. **Review recent earnings call transcripts:** Look for any discussion of progress towards those goals, including any challenges or opportunities.
3. **Analyze financial performance:** Assess Apple's recent revenue, profit, and other key metrics to see if they are trending in a direction that supports achieving their goals.
4. **Consider industry trends and market conditions:** The broader economic environment and technology industry trends can also impact Apple's ability to reach its goals.

**Remember:** 
* Financial data and market conditions are constantly changing. 
* It's best to consult reliable sources, such as financial news outlets and

In [43]:
query = "Based on recent earning call is the company apple on track to reach its 2024 goals?"
response_without_context, response_with_context, context = compare_added_context_through_rag(query, enhanced_vector_store, log=True)

response without context:

I do not have access to real-time information, including recent earnings calls or specific financial goals set by Apple for 2024. 

To determine if Apple is on track to reach its 2024 goals, you would need to:

1. **Access Apple's most recent earnings call transcript:** This can be found on the Apple Investor Relations website or through financial news sources.
2. **Identify Apple's stated goals for 2024:** Look for specific financial targets, revenue projections, or other key performance indicators mentioned by Apple's management.
3. **Analyze Apple's performance:** Compare Apple's actual results from the recent earnings call with their stated goals.  Consider factors like revenue growth, product sales, and operating margins.
4. **Consider external factors:** Analyze the broader economic climate, industry trends, and potential challenges that could affect Apple's progress.

Remember that financial projections are subject to change based on various factors. I

### Local GraphRAG

## Overview of Apple's Recent Earnings Call

During the Q3 2024 Earnings Conference Call held on August 1, 2024, Apple Inc. reported a revenue record of $85.8 billion for the June quarter, marking a 5% increase from the previous year. This performance exceeded expectations and was accompanied by a double-digit growth in earnings per share (EPS), which reached $1.40. The company also set revenue records in over two dozen countries, indicating strong global demand for its products and services [Data: Sources (53); Reports (10)].

## Key Financial Highlights

1. **Revenue Growth**: Apple's revenue growth was driven by its Services segment, which saw a 14% increase, achieving an all-time revenue record. This segment includes offerings such as iCloud, Apple Music, and the App Store, which are crucial for the company's overall financial health [Data: Reports (10); Relationships (288)].

2. **Product Performance**: The iPhone revenue was reported at $39.3 billion, although it experienced a slight decline of 1% year-over-year. However, on a constant currency basis, growth was noted, suggesting resilience in demand despite market fluctuations [Data: Sources (53)].

3. **Regional Performance**: In Europe, Apple reported a 1% increase in net sales for the second quarter of 2024 compared to the same period in 2023, while the Americas saw a slight decline of 1%. Notably, the Services segment performed well in Europe, contributing positively to overall sales [Data: Entities (125, 153); Relationships (288)].

## Strategic Initiatives and Future Outlook

Apple's commitment to innovation remains strong, with significant investments in research and development, particularly in areas like artificial intelligence and the upcoming Vision Pro headset. These initiatives are expected to enhance user engagement and expand market reach, aligning with the company's long-term growth strategy [Data: Reports (10); Sources (53)].

Tim Cook, Apple's CEO, expressed optimism about the company's future, emphasizing the transformative potential of new technologies and the ongoing expansion of its product ecosystem. The introduction of Apple Intelligence and the Vision Pro headset are seen as pivotal in maintaining Apple's competitive edge in the technology sector [Data: Sources (53)].

## Conclusion

Based on the insights from the recent earnings call, Apple appears to be on track to meet its 2024 goals. The combination of strong revenue growth, a robust Services segment, and strategic investments in innovation positions the company favorably for continued success. However, ongoing monitoring of market conditions and product performance will be essential to ensure that these goals are fully realized [Data: Reports (10); Sources (53)].

### GLobal GraphRAG

## Overview of Apple's 2024 Goals

Based on the recent Q3 2024 Earnings Conference Call, Apple Inc. appears to be on a positive trajectory towards achieving its 2024 goals. The company's executives, including CEO Tim Cook, expressed optimism regarding financial performance and strategic initiatives that are crucial for future growth.

## Financial Performance

Apple reported strong financial results for the June quarter, with record revenue primarily driven by significant sales of the iPhone and services. This robust performance is essential for maintaining investor confidence and supports the company's growth initiatives moving forward [Data: Reports (27, 28, 12)]. The Services segment, in particular, has shown strong revenue growth, which is critical for Apple's overall financial health [Data: Reports (28, 30, 12, 24, 11)].

## Strategic Focus and Innovation

The company's strategic focus on expanding its services is a vital component of its revenue generation strategy. Analysts noted that Apple's ongoing investment in Research and Development (R&D) is a key driver of innovation, which is necessary for maintaining its competitive edge [Data: Reports (24, 12)]. Additionally, initiatives like Apple Intelligence are expected to enhance user experience and drive revenue growth within the Services segment [Data: Reports (26)].

## Challenges and Risks

Despite the positive outlook, Apple faces challenges that may impact its growth trajectory. Market conditions in regions such as Greater China and Japan could pose risks to overall growth [Data: Reports (36, 37, 26)]. Furthermore, the company is currently dealing with multiple legal challenges, including antitrust lawsuits, which may affect its operational capabilities and market strategies [Data: Reports (21, 29)].

## Conclusion

In summary, Apple Inc. is on a solid path towards achieving its 2024 goals, bolstered by strong financial performance and strategic initiatives. However, the company must navigate market challenges and legal risks to maintain this momentum. The commitment to innovation and service expansion shall be critical in ensuring that Apple meets its objectives for the upcoming year [Data: Reports (21, 27, 28, 12)].